# Westeros Tutorial - Adding representation of renewables (part2/3): Introducing `"flexible generation"`

This tutorial, which demonstrates how to apply various model features to provide a more realistic representation of renewable energy integration in the energy system, is comprised of three parts. 

In the first part, we introduced [`"firm capacity"`](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html?highlight=FIRM_CAPACITY_PROVISION#equation-firm-capacity-provision) constraints to ensure that conventional electricity generation plants supplied sufficient backup capacity to allow for a high share of renewable electricity generation. In this tutorial, we will address [`"flexible generation"`](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html?highlight=flexibility#equation-system-flexibility-constraint) i.e., the ability of a power plant to ramp up and down its generation in response to the system needs. The power system needs to be flexible to respond to fluctuations in both electricity load and supply.

Further information can be found in [Sullivan et al., 2013](https://doi.org/10.1016/j.esr.2013.01.001).

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros scenario which adds emission taxes (``westeros_emissions_taxes.ipynb``) and solved it successfully

## Online documentation

The full framework documentation is available at [https://docs.messageix.org](https://docs.messageix.org).

In [ ]:
import ixmp

mp = ixmp.Platform()

## Load an existing scenario and clone it to a new scenario
We load the existing scenario '*carbon_tax*' and clone to a new scenario called '*flexible_generation*', to which we will apply the `"flexible generation"` constraint.

In [ ]:
from message_ix import Scenario

model = "Westeros Electrified"
base = Scenario(mp, model=model, scenario="carbon_tax")
scen = base.clone(
    model,
    "flexible_generation",
    "illustration of flexible-generation formulation",
    keep_solution=False,
)
scen.check_out()

## Retrieve parameters
We will retrieve those parameters necessary to perform subsequent additions of parameters.

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

## Describing Flexibility Requirements

Electric-sector flexibility is the ability of a power system to match supply and demand at any time scale. This characteristic can be represented in a stylized way as follows: 

Each generating technology is assigned a coefficient between −1 and 1 representing (if positive) the fraction of generation from that technology that is considered to be flexible or (if negative) the additional flexible generation required for each unit of generation from that technology. A negative coefficient on load reveals a parameterization of the amount of flexible energy the system requires solely to meet changes and uncertainty in load. ([Sullivan et al., 2013](https://doi.org/10.1016/j.esr.2013.01.001))

The coefficients used in this tutorial are derived in [Sullivan et al., 2013](https://doi.org/10.1016/j.esr.2013.01.001).

| Technology | Flexibility parameter |
| :--------- | :-------------------- |
| Load | −0.1 |
| Wind | −0.08 |
| Solar PV | −0.05 |
| Geothermal | 0 |
| Nuclear | 0 |
| Coal | 0.15 |
| Biopower | 0.3 |
| Gas-CC | 0.5 |
| Hydropower | 0.5 |
| H2 Electrolysis | 0.5 |
| Oil/gas steam | 1 |
| Gas-CT | 1 |
| Electricity storage | 1 |

Based on the above listed coefficients, our `"wind_ppl"` will need flexibility equal to 8% of its activity. Likewise, the electricity grid has a flexibility need of 10%. `"coal_ppl"` can provide 15% of its activity as flexibility for meeting the system needs.  

Recall that in the previous tutorial two `"rating_bin"`s were introduced for `"wind_ppl"`, depicting the different firm capacity requirements as market share increases.  We will again use the two rating bins previously defined. Here we make the assumption that the above flexibility parameter (8%) applies to the larger of the two rating bins (`"r2"`), and assume that the smaller rating bin (`"r1"`) has a lower flexibility demand of 6%, correlating to the fact that the first 20% of installed capacity required contributes more to firm capacity.  

### Add and parametrize rating bins
We will add the two rating bins and configure these for the `"wind_ppl"` as described above.

In [ ]:
from message_ix import make_df

scen.add_set("rating", ["r1", "r2"])

# Create the base dictionary for "rating_bin" parameter
base_rating = dict(
    node=country,
    commodity="electricity",
    level="secondary",
    unit="-",
    time="year",
    year_act=model_horizon,
)

name = "rating_bin"

rating_bin = make_df(name, **base_rating, technology="wind_ppl", value=0.2, rating="r1")
scen.add_par(name, rating_bin)

rating_bin = make_df(name, **base_rating, technology="wind_ppl", value=0.8, rating="r2")
scen.add_par(name, rating_bin)

### Add flexibility
We will configure the flexibility for the three technologies mentiond above: `"grid"`, `"wind_ppl"` and the `"coal_ppl"`.

In [ ]:
# Create the base dictionary for "flexibility_factor" parameter
base_flexibility_factor = dict(
    node_loc=country,
    commodity="electricity",
    level="secondary",
    mode="standard",
    unit="-",
    time="year",
    year_vtg=vintage_years,
    year_act=act_years,
)

name = "flexibility_factor"

# For the load (`grid`)
flexibility_factor = make_df(
    name, **base_flexibility_factor, technology="grid", rating="unrated", value=-0.1
)
scen.add_par(name, flexibility_factor)

# For the respective rating bins of `wind_ppl`
flexibility_factor = make_df(
    name, **base_flexibility_factor, technology="wind_ppl", rating="r1", value=-0.06
)
scen.add_par(name, flexibility_factor)

flexibility_factor = make_df(
    name, **base_flexibility_factor, technology="wind_ppl", rating="r2", value=-0.08
)
scen.add_par(name, flexibility_factor)

# For `coal_ppl`
flexibility_factor = make_df(
    name, **base_flexibility_factor, technology="coal_ppl", rating="unrated", value=0.15
)
scen.add_par(name, flexibility_factor)

Note that `"coal_ppl"` has a dynamic growth constraint on activity. This constraint will render the problem infeasible due to the flexibility requirements, hence we will remove this constraint.

In [ ]:
df = scen.par("growth_activity_up", filters={"technology": "coal_ppl", "year_act": 700})
scen.remove_par("growth_activity_up", df)

## Commit and solve

In [ ]:
scen.commit(comment="define parameters for flexibile-generation implementation")
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var("OBJ")["lvl"]

## Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

rep_base = Reporter.from_scenario(base)
prepare_plots(rep_base)

rep_scen = Reporter.from_scenario(scen)
prepare_plots(rep_scen)

### Activity
***
The impact of adding the flexibility constraint to electricity generation technologies on the model behavior can be observed by comparing the activity levels between the two scenarios. In the '*carbon_tax*' scenario, from which we started, electricity generation from `"wind_ppl"` is significantly scaled up over the years. By 720, almost all the electricity is generated by the `"wind_ppl"`.

When adding the flexibility constraint, the `"coal_ppl"` is required to generate a larger portion of electricity in order to provide activity to meet the flexibility demanded by the "load" and the `"wind_ppl"`.

#### Scenario: '*carbon_tax*'

In [ ]:
rep_base.set_filters(t=["coal_ppl", "wind_ppl"])
rep_base.get("plot activity")

#### Scenario: '*flexible_generation*'

In [ ]:
rep_scen.set_filters(t=["coal_ppl", "wind_ppl"])
rep_scen.get("plot activity")

### Impact on emissions
***
As a further consequence, the `"carbon_tax"`, which remains unchanged, does not result in the same emission reductions as achieved in the '*carbon_tax*' scenario (shown below). We can see that the emissions in 720 increase almost fivefold. Hence, adding the flexibility requirements to the scenario will require the `"carbon_tax"` to be increased  significantly to achieve the same emission reductions. Alternatively, a technology which contributes more of its activity to the flexibility constraint, like a gas combustion turbine, could be added to the model.

#### Scenario: '*carbon_tax*'

In [ ]:
base.var("EMISS", filters={"node": "Westeros"})

#### Scenario: '*flexible_generation*'

In [ ]:
scen.var("EMISS", filters={"node": "Westeros"})

## Price
***
Prices of electricity increase most notably in 720, as can be seen from the two plots below comparing the prices of `"light"`. Note that in 700, the price of `"light"` in the *'flexible_generation'* scenario is lower than the price in the *'carbon_tax'* scenario. This is because we have removed the dynamic growth constraint for `"coal_ppl"` in that year, allowing more electricity to be produced by the `"coal_ppl"` as opposed to the more expensive `"wind_ppl"`.

### Scenario: '*carbon_tax*'

In [ ]:
rep_base.set_filters(t=None, c=["light"])
rep_base.get("plot prices")

### Scenario: '*flexible_generation*'

In [ ]:
rep_scen.set_filters(t=None, c=["light"])
rep_scen.get("plot prices")

In [ ]:
mp.close_db()